In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Loading the DataFrame from CSV

In [2]:
df = pd.read_csv('1651489031.csv', index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        400 non-null    object
 1   company          400 non-null    object
 2   location         400 non-null    object
 3   compensation     20 non-null     object
 4   desc             363 non-null    object
 5   Seniority level  334 non-null    object
 6   Employment type  363 non-null    object
 7   Job function     334 non-null    object
 8   Industries       325 non-null    object
dtypes: object(9)
memory usage: 31.2+ KB


#### Dropping the unnnecessary columns
- Those withoud descriptions are basically irrelevant data.
- There may be some Functions and/or Industries that are blank, remove those as well.
- These are all entry level positions. Seniority Level can be dropped, or just fill the NaN cells with 'Entry Level'

In [4]:
df.dropna(subset=['desc', 'Job function', 'Industries'], inplace=True)
# df['Seniority level'].fillna('Entry Level', inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 0 to 399
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        325 non-null    object
 1   company          325 non-null    object
 2   location         325 non-null    object
 3   compensation     19 non-null     object
 4   desc             325 non-null    object
 5   Seniority level  325 non-null    object
 6   Employment type  325 non-null    object
 7   Job function     325 non-null    object
 8   Industries       325 non-null    object
dtypes: object(9)
memory usage: 25.4+ KB


#### Here are some available Job Functions on LinkedIn for Easier Categorization.
- New column for the Functions are added, and redundant columns were removed as well
- Separate salary ranges in 'upper' and 'lower' columns, fill all NaN with zeroes.
- Rearrange the table

In [6]:
functions = [
'Analyst',
'Accounting/Auditing', 
'Administrative', 
'Arts and Design', 
'Business',
'Social Services',
'Consulting',
'Education',
'Engineering',
'Entrepreneurship',
'Finance',
'Healthcare Services',
'Human Resources',
'Information Technology',
'Legal',
'Marketing',
'Media and Communication',
'Military',
'Operations',
'Product Management',
'Project Management',
'Purchasing',
'Quality Assurance',
'Real Estate',
'Research',
'Sales',
'Support'
]

In [7]:
# Input would be from each Job Functions column

def match_func(s):  
    # Split the job func using the comman and the 'and' delimiter
    if ',' in s:
        func_split = s.split(', ')
        func_split = [x.replace('and ', '') for x in func_split]
    else:
        func_split = s.split(' and ')

    
    # Loop through the func_split array, and return the first Function that matches the masterlist
    for key in func_split:
        for func in functions:
            if key.strip().lower() in func.lower():
                return func
    return key      # uncategorizeable job functions
        

df['Industry'] = df['Job function'].apply(lambda x: match_func(x))

In [8]:
# Remove non-legible records
import string
allowed_chars = string.printable

for row in df.index.values:
    if df.loc[row, 'job_title'][-2] not in allowed_chars:
        df.drop(row, inplace=True)


# Clean the location column and get the country

df['Country'] = df.location.apply(lambda x: x.split(', ')[-1])
df['Country'] = df['Country'].apply(lambda x: 'United States' if len(x) <= 2 or 'area' in x.lower() or 'metro' in x.lower() else x)

In [10]:
df['compensation'].fillna(0, inplace=True)

def clean_comp(c):
    if c == 0 or c == '':
        return 0, 0
    else:
        c = c.replace('$', '').replace(',','').replace('.00','')
        lower = c.split(' - ')[0].replace('/yr', '')
        upper = c.split(' - ')[1].replace('/yr', '')

        # convert all hourly figures to yearly
        if 'hr' in lower.lower() or 'hr' in upper.lower():
            lower = int(lower.replace('/hr', '')) * 2000
            upper = int(upper.replace('/hr', '')) * 2000

        return lower, upper

df.drop(df[df['compensation'].str.contains('CN') == True].index, inplace=True)
df['lower'] = df['compensation'].apply(lambda x: clean_comp(x)[0]).astype(float)
df['upper'] = df['compensation'].apply(lambda x: clean_comp(x)[1]).astype(float)

df = df.iloc[:, [5, 6, 9, 10, 11, 12]]

In [14]:
df.iloc[:, 5].value_counts()

0.0         298
119000.0      1
75000.0       1
61000.0       1
52000.0       1
54000.0       1
70000.0       1
92000.0       1
164000.0      1
69000.0       1
172000.0      1
101000.0      1
121000.0      1
Name: upper, dtype: int64

#### Some questions for the available data:
1. What function/Industry does recent job postings come from?
2. Which country is the most active in posting [xxxxx] related job in the past week?
3. Using those limited sample data with salary range, what would be the average/median salary for [xxxxx]?
    - What country is open to disclosing salary range in their job postings?
    - What industies?

In [ ]:
jobs_with_salary = df[df.lower != 0]
jobs_with_salary.head()